In [1]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
import ast
import os
from PIL import Image
import yaml

In [2]:
# CSV 파일 로드


def load_data_with_pandas(file_path):
    data = pd.read_csv(file_path)
    return data


# 데이터 로드
data_bumper = load_data_with_pandas(
    './data/data/output/bumper/bumper_data.csv')
data_door_ed = load_data_with_pandas(
    './data/data/output/door/door_ed_data.csv')
data_door_scratch = load_data_with_pandas(
    './data/data/output/door/door_scratch_data.csv')
data_fender = load_data_with_pandas(
    './data/data/output/fender/fender_ed_data.csv')
data_frame_ed = load_data_with_pandas(
    './data/data/output/frame/frame_ed_data.csv')
data_frame_hd = load_data_with_pandas(
    './data/data/output/frame/frame_hd_data.csv')
data_frame_sealf = load_data_with_pandas(
    './data/data/output/frame/frame_sealf_data.csv')
data_frame_seamf = load_data_with_pandas(
    './data/data/output/frame/frame_seamf_data.csv')

# 데이터 결합
data = pd.concat([
    data_bumper,
    data_door_ed,
    data_door_scratch,
    data_fender,
    data_frame_ed,
    data_frame_hd,
    data_frame_sealf,
    data_frame_seamf
], axis=0)

# data.to_csv('./data/data.csv', index=False)

# 학습 데이터와 검증 데이터로 나누기 (Stratified Sampling)
# y에는 라벨, x에는 나머지 특성 정보
# x = data.drop('quality', axis=1)  # 'label' 컬럼은 실제 클래스 라벨이 들어있는 컬럼
y = data['quality']

# Stratified Split: 학습 데이터와 검증 데이터 비율을 80:20으로 설정, 클래스 비율을 유지
train_data, valid_data = train_test_split(
    data, test_size=0.2, stratify=y, random_state=55)

# CSV로 저장
train_data.to_csv('./data/train_data.csv', index=False)
valid_data.to_csv('./data/valid_data.csv', index=False)

# 결과 출력
print(f"학습 데이터 개수: {len(train_data)}")
print(f"검증 데이터 개수: {len(valid_data)}")

학습 데이터 개수: 5904
검증 데이터 개수: 1477


In [3]:
# Step 1: 학습 데이터에서 20% 추출
train_20, _ = train_test_split(train_data, test_size=0.8, random_state=35)

# Step 2: 검증 데이터에서 20% 추출
valid_20, _ = train_test_split(valid_data, test_size=0.8, random_state=35)

# Step 3: 각각의 20%에서 다시 20% 추출 (성능 검증용 데이터 생성)
train_test_20, _ = train_test_split(train_20, test_size=0.8, random_state=42)  # 학습 데이터 20%에서 20% 추출
valid_test_20, _ = train_test_split(valid_20, test_size=0.8, random_state=42)  # 검증 데이터 20%에서 20% 추출

# CSV로 저장
train_test_20.to_csv('./data/train_test_20.csv', index=False)
valid_test_20.to_csv('./data/valid_test_20.csv', index=False)

In [4]:
# 이미지 복사
import shutil

def copy_images(data, output_dir):
    image_dir = os.path.join(output_dir, 'images')
    os.makedirs(image_dir, exist_ok=True)

    for index, row in data.iterrows():
        image_path = row['path']
        shutil.copy(image_path, image_dir)


# 학습 데이터와 검증 데이터를 각각 처리
copy_images(train_test_20, './train')
copy_images(valid_test_20, './valid')

In [5]:
import ast

def parse_bboxes(data):
    """
    문자열 형태의 bounding box 정보를 파싱하여 리스트 형태로 변환하는 함수

    Args:
        data: 파싱할 데이터셋

    Returns:
        list: 파싱된 bounding box 리스트
    """

    bboxes_list = []
    for bbox_str in data["bboxes"]:
        try:
            bboxes = ast.literal_eval(bbox_str)
            bbox_list = []
            for bbox in bboxes:
                x, y, w, h = bbox
                bbox_list.append([x, y, w, h])
            bboxes_list.append(bbox_list)
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing bbox: {e}")
    return bboxes_list

# 데이터셋별 bounding box 파싱
train_bboxes = parse_bboxes(train_test_20)
valid_bboxes = parse_bboxes(valid_test_20)

# 결과 출력 (예시)
print(train_bboxes[:5])
print(valid_bboxes[:5])

[[[2305.3953488372094, 644.6511627906976, 558.6976744186047, 233.30232558139537]], [[1062.7009664036666, 929.466618422309, 1754.4165579313199, 255.92260313457723]], [[1488.5581395348836, 1012.5276073619633, 808.7441860465119, 251.6687116564416]], [], [[1483.439402095895, 918.8834355828221, 479.81641185759327, 134.6134969325151]]]
[[[1780.3282928673666, 702.7805816986404, 604.8810094582147, 210.7061323385657]], [[1760.4836851607922, 678.9202453987729, 996.2790697674418, 298.49079754601223]], [[1594.046511627907, 784.9806451612903, 720.8372093023255, 193.31612903225806]], [], [[946.7289634630656, 1612.6145009810286, 152.71064846121396, 162.55190492641364]]]


In [6]:
import os
from PIL import Image


def extract_image_info(image_paths):
    """
    이미지 경로 리스트에서 이미지 크기, 디렉토리 경로, 파일 이름을 추출하는 함수

    Args:
        image_paths (list): 이미지 경로 리스트

    Returns:
        tuple: (image_sizes, image_dirs, file_names)
            image_sizes (list): 이미지 크기 리스트
            image_dirs (list): 이미지 디렉토리 경로 리스트
            file_names (list): 이미지 파일 이름 리스트
    """

    image_sizes = []
    image_dirs = []
    file_names = []

    for path in image_paths:
        try:
            image = Image.open(path)
            width, height = image.size
            image_sizes.append((width, height))
            image_dirs.append(os.path.dirname(path) + os.sep)
            file_names.append(os.path.splitext(os.path.basename(path))[0])
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path}")

    return image_sizes, image_dirs, file_names


# 데이터 준비 (예시)
train_image_paths = train_test_20.iloc[:, 0].tolist()
valid_image_paths = valid_test_20.iloc[:, 0].tolist()

# 이미지 정보 추출
train_sizes, train_dirs, train_names = extract_image_info(train_image_paths)
valid_sizes, valid_dirs, valid_names = extract_image_info(valid_image_paths)

# 결과 출력
print("Train 이미지 크기:", train_sizes[:5])
print("Train 이미지 디렉토리:", train_dirs[:5])
print("Train 파일 이름:", train_names[:5])
print()
print("Validation 이미지 크기:", valid_sizes[:5])
print("Validation 이미지 디렉토리:", valid_dirs[:5])
print("Validation 파일 이름:", valid_names[:5])

Train 이미지 크기: [(4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000)]
Train 이미지 디렉토리: ['./data/data/source_data/door/Exterior_damage\\', './data/data/source_data/frame/Seam_failure\\', './data/data/source_data/bumper/scratch\\', './data/data/source_data/door/scratch\\', './data/data/source_data/bumper/scratch\\']
Train 파일 이름: ['204_102_20_6d79e1bc-442e-45d6-bd7f-a09baeefee49', '207_212_20_33af8e26-7021-421c-88fe-683c37c5e565', '205_101_20_d7504074-f53e-4c44-84ec-636e61f902ac', '204_101_10_5620de38-ef45-4fb1-a051-4067b7704f1c', '205_101_20_05b4b1a5-6a7a-4e68-bdb5-17316ae94eb4']

Validation 이미지 크기: [(4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000)]
Validation 이미지 디렉토리: ['./data/data/source_data/door/Exterior_damage\\', './data/data/source_data/frame/Seam_failure\\', './data/data/source_data/frame/Seam_failure\\', './data/data/source_data/door/scratch\\', './data/data/source_data/frame/Hole_distortion\\']
Validation 파일 이름: ['204_102_20_019dc27d-2034-44b5

In [7]:
# 부품 상태를 체크하는 함수
def get_label(part, status):
    if part == "범퍼":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    elif part == "도어":
        if status == 1:
            return 2
        elif status == 0:
            return 3
    elif part == "휀더":
        if status == 1:
            return 4
        elif status == 0:
            return 5
    elif part == "프레임":
        if status == 1:
            return 6
        elif status == 0:
            return 7
    return None  # 기본적으로 반환할 값이 없으면 None


# 예시 부품과 상태
parts_and_status_train = []
parts_and_status_valid = []
for quality, part in zip(train_test_20.iloc[:, 1], train_test_20.iloc[:, 2]):
    parts_and_status_train.append({"part": part, "status": quality})

for quality, part in zip(valid_test_20.iloc[:, 1], valid_test_20.iloc[:, 2]):
    parts_and_status_valid.append({"part": part, "status": quality})

print(f"parts_and_status_train : {parts_and_status_train}")
print(f"parts_and_status_valid : {parts_and_status_valid}")

# 라벨링 결과를 저장할 리스트
labels_train = []
labels_valid = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train:
    label = get_label(item["part"], item["status"])
    labels_train.append(label)
for item in parts_and_status_valid:
    label = get_label(item["part"], item["status"])
    labels_valid.append(label)

# # 결과 출력
print("train 라벨링 결과 :", labels_train)
print("valid 라벨링 결과 :", labels_valid)

parts_and_status_train : [{'part': '도어', 'status': 0}, {'part': '프레임', 'status': 0}, {'part': '범퍼', 'status': 0}, {'part': '도어', 'status': 1}, {'part': '범퍼', 'status': 0}, {'part': '범퍼', 'status': 0}, {'part': '도어', 'status': 1}, {'part': '프레임', 'status': 0}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 0}, {'part': '도어', 'status': 1}, {'part': '범퍼', 'status': 0}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 0}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '도어', 'status': 0}, {'part': '범퍼', 'status': 0}, {'part': '범퍼', 'status': 0}, {'part': '프레임', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '프레임', 'status': 0}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 0}, {'part': '프레임', 'status': 0}, {'part': '도어', 'status': 1}, {'part': '프레임', 'status': 0}, {'part': '프레임', 'status': 0}, {'part': '도어', 'status': 0}, {'part': '휀더', 'status': 1}, {'

In [8]:
def convert_to_yolo_format(bbox, label, image_width, image_height):
    yolo_labels = []  # 각 이미지에 대한 라벨을 담을 리스트

    if not bbox:
        # bbox가 비어있으면 기본값으로 라벨을 설정
        yolo_labels.append(f"{label} 0.5 0.5 0.0 0.0")
    else:
        # bbox가 있으면 여러 개의 파손 부위를 처리
        for x_min, y_min, width, height in bbox:
            # YOLO 형식에 맞게 좌표를 정규화
            x_center = (x_min + width / 2) / image_width
            y_center = (y_min + height / 2) / image_height
            norm_width = width / image_width
            norm_height = height / image_height
            # 정규화된 좌표와 함께 라벨을 저장
            yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {
                               norm_width:.6f} {norm_height:.6f}")

    return yolo_labels


def save_yolo_labels(output_dir, bboxes, labels, size, names):
    # 각 이미지에 대해 라벨을 텍스트 파일로 저장
    for bbox, label, (w, h), name in zip(bboxes, labels, size, names):
        yolo_labels = convert_to_yolo_format(bbox, label, w, h)

        # 라벨을 .txt 파일로 저장
        txt_file_path = os.path.join(output_dir, f"{name}.txt")
        with open(txt_file_path, 'w') as txt_file:
            for yolo_data in yolo_labels:
                txt_file.write(f"{yolo_data}\n")



# 라벨을 저장할 디렉토리
train_output_dir = './train/labels'
os.makedirs(train_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(train_output_dir, train_bboxes, labels_train, train_sizes, train_names)

# 라벨을 저장할 디렉토리
valid_output_dir = './valid/labels'
os.makedirs(valid_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(valid_output_dir, valid_bboxes, labels_valid, valid_sizes, valid_names)

In [9]:
# YAML 파일에 저장
yaml_data = {
    "train": "C:/Users/Administrator/sf_project_2/train/images",
    "val": "C:/Users/Administrator/sf_project_2/valid/images",
    "nc": 8,
    "names": ["bumper_ok",
              "bumper_faulty",
              "door_ok",
              "door_faulty",
              "fender_ok",
              "fender_faulty",
              "frame_ok",
              "frame_faulty"]
}

with open(f"data.yaml", "w") as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)

# print("YAML 파일이 생성되었습니다.")

In [10]:
'''
1. yolo 데이터 돌려보기
2. test데이터 만들기
3. openCV 연결 -> 실험
4. PPT 작성

'''

'\n1. yolo 데이터 돌려보기\n2. test데이터 만들기\n3. openCV 연결 -> 실험\n4. PPT 작성\n\n'

In [ ]:

import os
from ultralytics import YOLO

# 데이터 설정 파일 경로
data_yaml = "data.yaml"

# YOLO 모델 정의 (랜덤 초기화로 학습 시작)
model = YOLO("yolov8n.pt")  # 'yolov5n'은 작은 모델, 필요에 따라 yolov5s, yolov5m 선택 가능
model.model = model.model.reset_parameters()  # 가중치 초기화 (랜덤 가중치)

# 학습 설정
model.train(
    data=data_yaml,           # 데이터 설정 파일 경로
    epochs=100,               # 학습 에폭 수
    batch_size=16,            # 배치 크기
    imgsz=640,                # 입력 이미지 크기
    weights=None,             # 사전 학습 가중치 사용 안 함
    device=0,                 # GPU 사용
    workers=4,                # 데이터 로드 병렬 처리 워커 수
    project="runs/train",     # 결과 저장 디렉토리
    name="custom_model"       # 실험 이름
)

# 모델 로드 및 추론
model = YOLO("runs/train/custom_model/weights/model.pt")
results = model.predict(source="path/to/test_images", save=True, imgsz=640)

In [ ]:

from ultralytics import YOLO

# Load a pre-trained YOLO model (you can choose n, s, m, l, or x versions)
model = YOLO("yolov8n.pt")  # 사전 학습된 YOLO 모델

# 학습 수행
model.train(
    data="data.yaml",  # yaml 파일 경로
    epochs=10,         # 학습 에포크
    imgsz=640,         # 이미지 크기
    batch=8,          # 배치 크기
    device=0           # GPU 장치
)



New https://pypi.org/project/ultralytics/8.3.64 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.63  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning C:\Users\Administrator\sf_project_2\train\labels.cache... 236 images, 0 backgrounds, 10 corrupt: 100%|██████████| 236/236 [00:00<?, ?it/s]

train: WARNING  C:\Users\Administrator\sf_project_2\train\images\204_101_20_77def2bc-7e02-4ed6-b544-2d8080a2c794.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.00214 1.38307]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\205_101_20_0f7beccd-2c7d-405a-810b-12a168c587a4.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.066605]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\205_101_20_15165e7e-6348-4728-94c4-595f7c407b00.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.093452]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\205_101_20_37ba50f3-7010-407c-879f-67b6da4705d7.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.011244]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\205_101_20_79ad1afb-2fdc-4060-91f4-3f64a6684902.jpg: ignoring corrupt image/label: non-normalized or out of bounds c


val: Scanning C:\Users\Administrator\sf_project_2\valid\labels.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]


Plotting labels to c:\Users\Administrator\opencv\runs\detect\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\Users\Administrator\opencv\runs\detect\train7
Starting training for 10 epochs...
Closing dataloader mosaic
